In [0]:
from pyspark.sql import functions as F
from pyspark.sql.types import LongType, StringType, BooleanType, TimestampType
from delta.tables import DeltaTable
from datetime import datetime

CATALOGO_ORIGEM = "v_credit"
SCHEMA_ORIGEM = "bronze"
TABELA_ORIGEM = "canais"

CATALOGO_DESTINO = "v_credit"
SCHEMA_DESTINO = "silver"
TABELA_DESTINO = "tb_canal"

nome_tabela_origem = f"{CATALOGO_ORIGEM}.{SCHEMA_ORIGEM}.{TABELA_ORIGEM}"
nome_tabela_destino = f"{CATALOGO_DESTINO}.{SCHEMA_DESTINO}.{TABELA_DESTINO}"

timestamp_atual = F.current_timestamp()


In [0]:
canais_bronze_df = spark.read.table("v_credit.bronze.canais")

ativos_variacoes = ["ativo", "ativoo", "atvo", "atvivo", "atvio", "ativ", "atv", "at1vo", "a tivo"]
inativos_variacoes = ["inativo", "inatvo", "invativo", "inativoo", "inativ", "inativ0", "in ativo", "inat1vo", "in at1vo"]

canais_silver_df = (
    canais_bronze_df
    .withColumn("dt_ingestion", timestamp_atual)
    .withColumnRenamed("nome_canal", "nm_canal")
    .withColumnRenamed("canal_status", "st_ativo_raw")
    .withColumnRenamed("origem", "dc_origem")
    .withColumn("nm_canal", F.col("nm_canal").cast(StringType()))
    .withColumn("st_ativo_raw", F.col("st_ativo_raw").cast(StringType()))
    .withColumn("dc_origem", F.col("dc_origem").cast(StringType()))
    .withColumn(
        "st_ativo",
        F.when(
            F.lower(F.col("st_ativo_raw")).isin([v.lower() for v in inativos_variacoes]),
            F.lit(False)
        ).when(
            F.lower(F.col("st_ativo_raw")).isin([v.lower() for v in ativos_variacoes]),
            F.lit(True)
        ).otherwise(F.lit(None))
    )
    .withColumn("nm_canal", F.upper(F.col("nm_canal")))
    .select(
        "nm_canal",
        "st_ativo",
        "st_ativo_raw",
        "dt_ingestion",
        "dc_origem"
    )
)

canais_validacao_df = (
    canais_silver_df
    .withColumn(
        "flag_qualidade",
        F.when(
            F.col("st_ativo").isNotNull(),
            F.lit("OK")
        ).otherwise(F.lit("ERRO"))
    )
)

canais_validos_df = (
    canais_validacao_df
    .filter(F.col("flag_qualidade") == "OK")
    .drop("flag_qualidade", "st_ativo_raw")
)

canais_invalidos_df = canais_validacao_df.filter(F.col("flag_qualidade") == "ERRO")

display(canais_validos_df)
display(canais_invalidos_df)

In [0]:
delta_table = DeltaTable.forName(spark, nome_tabela_destino)

delta_table.alias("destino").merge(
    canais_validos_df.alias("origem"),
    "destino.nm_canal = origem.nm_canal"
).whenMatchedUpdate(
    set = {
        "nm_canal": "origem.nm_canal", 
        "st_ativo": "origem.st_ativo",
        "dt_ingestion": "origem.dt_ingestion",
        "dc_origem": "origem.dc_origem"
    }
).whenNotMatchedInsert(
    values = {
        "nm_canal": "origem.nm_canal", 
        "st_ativo": "origem.st_ativo",
        "dt_ingestion": "origem.dt_ingestion",
        "dc_origem": "origem.dc_origem"
    }
).execute()